In [1]:
import os
import sagemaker
import numpy as np
from sagemaker.tensorflow import TensorFlow

In [2]:
ON_SAGEMAKER_NOTEBOOK = True

train_instance_type='ml.p2.xlarge'
deploy_instance_type='ml.p2.xlarge'

bucket = 'sagemaker-mumbai-data-set'
key = 'ena_data'

In [3]:
train_input_path = "s3://{}/{}/train/".format(bucket, key)
validation_input_path = "s3://{}/{}/validation/".format(bucket, key)

In [4]:
sagemaker_session = sagemaker.Session()
if ON_SAGEMAKER_NOTEBOOK:
    role = sagemaker.get_execution_role()
else:
    role = '[IAM ROLE]'

In [5]:
hyperparameters = {'epochs': 1, 'batch_size' : 64}

In [6]:
os.path.abspath('')

'/home/ec2-user/SageMaker/sagemaker_imageclassify/script_mode'

In [7]:
estimator = TensorFlow(base_job_name='ena-classify-inceptionv3-2',
                       entry_point=os.path.join(os.path.abspath(''),'entry.py'),
                       role=role,
                       framework_version='2.0.0',
                       py_version='py3',
#                        output_path='s3://sagemaker-mumbai-data-set/ena_data',
                       hyperparameters=hyperparameters,
                       train_instance_count=1, train_instance_type=train_instance_type)

In [8]:
print("Training ...")
local_inputs = {'train' : train_input_path, 
                'validation' : validation_input_path}
estimator.fit(local_inputs)

Training ...
2020-01-14 19:40:14 Starting - Starting the training job...
2020-01-14 19:40:16 Starting - Launching requested ML instances...
2020-01-14 19:41:12 Starting - Preparing the instances for training.........
2020-01-14 19:42:35 Downloading - Downloading input data......
2020-01-14 19:43:35 Training - Training image download completed. Training in progress..2020-01-14 19:43:39,147 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-01-14 19:43:39,646 sagemaker-containers INFO     Invoking user script

Training Env:

{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "validation": "/opt/ml/input/data/validation",
        "train": "/opt/ml/input/data/train"
    },
    "current_host": "algo-1",
    "framework_module": "sagemaker_tensorflow_container.training:main",
    "hosts": [
        "algo-1"
    ],
    "hyperparameters": {
        "batch_size": 64,
        "model_dir": "s3://sagemaker-mumbai-data-set/ena_

/usr/local/lib/python3.6/dist-packages/PIL/Image.py:989: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "
#015 1/63 [..............................] - ETA: 5:33 - loss: 1.7604 - accuracy: 0.2344#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#015 2/63 [..............................] - ETA: 3:03 - loss: 3.7562 - accuracy: 0.2656#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010#010

2020-01-14 19:50:23.108790: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model successfully saved at: /opt/ml/model
2020-01-14 19:50:46,436 sagemaker-containers INFO     Reporting training SUCCESS

2020-01-14 19:51:11 Uploading - Uploading generated training model
2020-01-14 19:51:11 Completed - Training job completed
Training seconds: 516
Billable seconds: 516


In [37]:
deploy_instance_type = 'ml.p2.xlarge'
predictor = estimator.deploy(initial_instance_count=1, instance_type=deploy_instance_type)

KeyError: 'ModelArtifacts'

In [38]:
predictor = estimator.deploy(initial_instance_count=1, instance_type='ml.m4.xlarge')

KeyError: 'ModelArtifacts'